### Eating Habits Analysis of KAIST Students (KAIST 학생들의 식습관 DB 분석)

In [ ]:
import pandas as pd

df = pd.read_excel("./database.xlsx", sheet_name=[0, 1, 2])
print(f"df: {df}")

#### Class Student and Class Meal

In [9]:
class Student:
    def __init__(self, 피험자번호, 성별, 연령, 체중유형, 거주유형, 전공영역, meals):
        self.피험자번호 = 피험자번호
        self.성별 = 성별
        self.연령 = 연령
        self.체중유형 = 체중유형
        self.거주유형 = 거주유형
        self.전공영역 = 전공영역
        self.meals = meals

    def __str__(self):
        id = "\n\n" + f"{self.피험자번호}"
        meals = [str(meal) for meal in self.meals]
        return "\n  ".join([id] + meals)

    def __repr__(self):
        return str(self)


class Meal:
    def __init__(
        self, 요일, 식사전스트레스지수, 식사유형, 식사위치, 음식유형, 가격, 식사만족도, 함께한사람, 이동거리, 식사시간, 이동수단
    ):
        self.요일 = 요일
        self.식사전스트레스지수 = 식사전스트레스지수
        self.식사유형 = 식사유형
        self.식사위치 = 식사위치
        self.음식유형 = 음식유형
        self.가격 = 가격
        self.식사만족도 = 식사만족도
        self.함께한사람 = 함께한사람
        self.이동거리 = 이동거리
        self.식사시간 = 식사시간
        self.이동수단 = 이동수단

    def __str__(self):
        return f"{self.식사유형}, {self.식사위치}, {self.가격}, {self.함께한사람}, {self.이동거리}, {self.식사시간}"


#### Load Data into Lists

In [11]:
# In 2018, 'Residential Type' was not recorded. Therefore, for the
# ease of processing, "미기록" is added for students in year 2018.

students_2018, students_2021, students_2022 = [], [], []

for key, sheet in df.items():
    if key == 0:
        students = students_2022
    elif key == 1:
        students = students_2021
    elif key == 2:
        students = students_2018
    boundary = 5 if key == 2 else 6

    for _, row in sheet.iterrows():
        data = row.to_dict()
        # The same student as the previous one
        if students and students[-1].피험자번호 == data.get("피험자번호"):
            student = students[-1]
        # New student
        else:
            characters = list(data.values())[:boundary]
            if boundary == 5:
                characters.insert(4, "미기록")
            student = Student(*characters, [])
            students.append(student)
        meal = Meal(*list(data.values())[boundary:])
        student.meals.append(meal)

#### Eating Habit DB Composition (식습관 DB 구성)

In [17]:
from collections import Counter

students = students_2022 + students_2021 + students_2018

genders = Counter([student.성별 for student in students])
age = Counter([student.연령 for student in students])
weight_type = Counter([student.체중유형 for student in students])
residential_type = Counter([student.거주유형 for student in students])
major = Counter([student.전공영역 for student in students])

print(f"2022: {len(students_2022)}명, 2021: {len(students_2021)}명, 2018: {len(students_2018)}명")
print(gender, age, weight_type, residential_type, major, sep="\n")

2022: 49명, 2021: 71명, 2018: 49명
Counter({'남성': 126, '여성': 43})
Counter({21: 41, 22: 38, 20: 30, 23: 25, 24: 19, 19: 9, 25: 7})
Counter({'표준체중': 127, '과체중': 24, '저체중': 18})
Counter({'기숙사': 75, '미기록': 49, '자취': 29, '본가': 16})
Counter({'공과대학': 141, '자연과학대학': 19, '생명과학기술대학': 7, '융합인재학부': 2})


#### Helper functions

In [62]:
def print_ratios(entries: list[str], counters: list[dict]):
    """Calculate and print ratios for each counter.

    Args:
        entries (list[str]): entries for a field.
            e.g. ["아침", "점심", "저녁", "간식", "야식", "음주"] for 'meal types' field.
        counters (list[dict]): three counters for year 2022, 2021, 2018.
    """
    totals = [sum(counter.values()) for counter in counters]
    for idx, counter in enumerate(counters):
        ratios = []
        for entry in entries:
            ratio = "%s: %5.2f%%" % (entry, counter[entry] / totals[idx] * 100)
            ratios.append(ratio)
        print(", ".join(ratios))


#### Changes in Meal Types (식사 유형의 변화)

In [63]:
counter_2022 = Counter([meal.식사유형 for student in students_2022 for meal in student.meals])
counter_2021 = Counter([meal.식사유형 for student in students_2021 for meal in student.meals])
counter_2018 = Counter([meal.식사유형 for student in students_2018 for meal in student.meals])

counter_2022, counter_2021, counter_2018 = dict(counter_2022), dict(counter_2021), dict(counter_2018)
print(counter_2022, counter_2021, counter_2018, sep="\n")

meal_types = ["아침", "점심", "저녁", "간식", "야식", "음주"]
print()
print_ratios(meal_types, [counter_2022, counter_2021, counter_2018])


{'저녁': 328, '야식': 73, '아침': 98, '점심': 303, '간식': 159, '음주': 40}
{'점심': 437, '간식': 225, '저녁': 480, '야식': 63, '음주': 38, '아침': 150}
{'저녁': 313, '야식': 83, '점심': 266, '간식': 148, '아침': 64, '음주': 22}

아침:  9.79%, 점심: 30.27%, 저녁: 32.77%, 간식: 15.88%, 야식:  7.29%, 음주:  4.00%
아침: 10.77%, 점심: 31.37%, 저녁: 34.46%, 간식: 16.15%, 야식:  4.52%, 음주:  2.73%
아침:  7.14%, 점심: 29.69%, 저녁: 34.93%, 간식: 16.52%, 야식:  9.26%, 음주:  2.46%


#### Changes in Meal Location (식사 위치의 변화)

In [65]:
counter_2022 = Counter([meal.식사위치 for student in students_2022 for meal in student.meals])
counter_2021 = Counter([meal.식사위치 for student in students_2021 for meal in student.meals])
counter_2018 = Counter([meal.식사위치 for student in students_2018 for meal in student.meals])

counter_2022, counter_2021, counter_2018 = dict(counter_2022), dict(counter_2021), dict(counter_2018)
print(counter_2022, counter_2021, counter_2018, sep="\n")

meal_locations = ["기숙사", "집", "북측", "동측", "서측", "어은동", "궁동", "둔산동", "대전기타", "대전외지역"]
print()
print_ratios(meal_locations, [counter_2022, counter_2021, counter_2018])


{'북측': 169, '어은동': 149, '기숙사': 199, '동측': 59, '대전기타': 32, '집': 162, '대전외지역': 117, '서측': 45, '궁동': 57, '둔산동': 12}
{'어은동': 162, '집': 511, '서측': 27, '대전기타': 69, '대전외지역': 122, '기숙사': 267, '동측': 30, '북측': 125, '둔산동': 12, '궁동': 68}
{'대전기타': 53, '어은동': 156, '둔산동': 32, '궁동': 58, '북측': 197, '동측': 73, '기숙사': 209, '서측': 35, '대전외지역': 41, '집': 42}

기숙사: 19.88%, 집: 16.18%, 북측: 16.88%, 동측:  5.89%, 서측:  4.50%, 어은동: 14.89%, 궁동:  5.69%, 둔산동:  1.20%, 대전기타:  3.20%, 대전외지역: 11.69%
기숙사: 19.17%, 집: 36.68%, 북측:  8.97%, 동측:  2.15%, 서측:  1.94%, 어은동: 11.63%, 궁동:  4.88%, 둔산동:  0.86%, 대전기타:  4.95%, 대전외지역:  8.76%
기숙사: 23.33%, 집:  4.69%, 북측: 21.99%, 동측:  8.15%, 서측:  3.91%, 어은동: 17.41%, 궁동:  6.47%, 둔산동:  3.57%, 대전기타:  5.92%, 대전외지역:  4.58%


#### Changes in Number of People Together (함께한 사람 수의 변화)

In [66]:
counter_2022 = Counter([meal.함께한사람 for student in students_2022 for meal in student.meals])
counter_2021 = Counter([meal.함께한사람 for student in students_2021 for meal in student.meals])
counter_2018 = Counter([meal.함께한사람 for student in students_2018 for meal in student.meals])

counter_2022, counter_2021, counter_2018 = dict(counter_2022), dict(counter_2021), dict(counter_2018)
print(counter_2022, counter_2021, counter_2018, sep="\n")

number_of_people = ["혼밥", "다밥", "회식"]
print()
print_ratios(number_of_people, [counter_2022, counter_2021, counter_2018])

{'혼밥': 444, '다밥': 522, '회식': 35}
{'혼밥': 684, '다밥': 693, '회식': 16}
{'다밥': 540, '혼밥': 327, '회식': 29}

혼밥: 44.36%, 다밥: 52.15%, 회식:  3.50%
혼밥: 49.10%, 다밥: 49.75%, 회식:  1.15%
혼밥: 36.50%, 다밥: 60.27%, 회식:  3.24%


#### Changes in Distance Traveled (이동 거리의 변화)

In [67]:
counter_2022 = Counter([meal.이동거리 for student in students_2022 for meal in student.meals])
counter_2021 = Counter([meal.이동거리 for student in students_2021 for meal in student.meals])
counter_2018 = Counter([meal.이동거리 for student in students_2018 for meal in student.meals])

counter_2022, counter_2021, counter_2018 = dict(counter_2022), dict(counter_2021), dict(counter_2018)
print(counter_2022, counter_2021, counter_2018, sep="\n")

distance_traveled = ["없음", "15분이내", "30분이내", "30분이상"]
print()
print_ratios(distance_traveled, [counter_2022, counter_2021, counter_2018])


{'없음': 436, '15분이내': 420, '30분이내': 78, '30분이상': 67}
{'15분이내': 445, '없음': 762, '30분이내': 123, '30분이상': 63}
{'15분이내': 463, '30분이내': 99, '없음': 293, '30분이상': 41}

없음: 43.56%, 15분이내: 41.96%, 30분이내:  7.79%, 30분이상:  6.69%
없음: 54.70%, 15분이내: 31.95%, 30분이내:  8.83%, 30분이상:  4.52%
없음: 32.70%, 15분이내: 51.67%, 30분이내: 11.05%, 30분이상:  4.58%


In [68]:
total_2022 = sum(counter_2022.values())
total_2021 = sum(counter_2021.values())
total_2018 = sum(counter_2018.values())

meal_types = [["없음", "15분이내"], ["30분이내", "30분이상"]]
ratio_2022, ratio_2021, ratio_2018 = [], [], []
for meal_type in meal_types:
    ratio = sum([counter_2022[key] for key in meal_type]) / total_2022
    ratio_2022.append("%s: %5.2f%%" % (meal_type, ratio * 100))
    ratio = sum([counter_2021[key] for key in meal_type]) / total_2021
    ratio_2021.append("%s: %5.2f%%" % (meal_type, ratio * 100))
    ratio = sum([counter_2018[key] for key in meal_type]) / total_2018
    ratio_2018.append("%s: %5.2f%%" % (meal_type, ratio * 100))

print("", ", ".join(ratio_2022), ", ".join(ratio_2021), ", ".join(ratio_2018), sep="\n")



['없음', '15분이내']: 85.51%, ['30분이내', '30분이상']: 14.49%
['없음', '15분이내']: 86.65%, ['30분이내', '30분이상']: 13.35%
['없음', '15분이내']: 84.38%, ['30분이내', '30분이상']: 15.62%


#### Changes in Duration (식사 시간의 변화)

In [71]:
counter_2022 = Counter([meal.식사시간 for student in students_2022 for meal in student.meals])
counter_2021 = Counter([meal.식사시간 for student in students_2021 for meal in student.meals])
counter_2018 = Counter([meal.식사시간 for student in students_2018 for meal in student.meals])

counter_2022, counter_2021, counter_2018 = dict(counter_2022), dict(counter_2021), dict(counter_2018)
print(counter_2022, counter_2021, counter_2018, sep="\n")

duration = ["15분이내", "30분이내", "30분이상"]
print()
print_ratios(duration, [counter_2022, counter_2021, counter_2018])


{'30분이내': 366, '15분이내': 359, '30분이상': 276}
{'30분이내': 430, '30분이상': 383, '15분이내': 580}
{'30분이상': 226, '30분이내': 274, '15분이내': 396}

15분이내: 35.86%, 30분이내: 36.56%, 30분이상: 27.57%
15분이내: 41.64%, 30분이내: 30.87%, 30분이상: 27.49%
15분이내: 44.20%, 30분이내: 30.58%, 30분이상: 25.22%


#### Changes in Price (가격의 변화)

In [54]:
def average(meals):
    return sum([meal.가격 for meal in meals])/len(meals)


meals_2022 = [meal for student in students_2022 for meal in student.meals]
meals_2021 = [meal for student in students_2021 for meal in student.meals]
meals_2018 = [meal for student in students_2018 for meal in student.meals]
print("2022: %6.2f원, 2021: %6.2f원, 2018: %6.2f원" % \
      (average(meals_2022), average(meals_2021), average(meals_2018)))


def price_with_loc(meals):
    house, campus, outside, daejeon = [], [], [], []
    for meal in meals:
        if meal.식사위치 in ['기숙사', '집']:
            house.append(meal)
        elif meal.식사위치 in ["북측", "동측", "서측"]:
            campus.append(meal)
        elif meal.식사위치 in ["어은동", "궁동", "둔산동", "대전기타", "대전외지역"]:
            outside.append(meal)
        if meal.식사위치 in ["어은동", "궁동", "둔산동", "대전기타"]:
            daejeon.append(meal)
    return average(house), average(campus), average(outside), average(daejeon)


house_2022, campus_2022, outside_2022, daejeon_2022 = price_with_loc(meals_2022)
house_2021, campus_2021, outside_2021, daejeon_2021 = price_with_loc(meals_2021)
house_2018, campus_2018, outside_2018, daejeon_2018 = price_with_loc(meals_2018)

print("\n[ 위치에 따른 식사 비용 ]")
print("방 (기숙사, 집) \t\t\t\t: %6.2f원 (2022), %6.2f원 (2021), %6.2f원 (2018)" % (house_2022, house_2021, house_2018))
print("교내 (동측, 서측, 북측) \t\t\t: %6.2f원 (2022), %6.2f원 (2021), %6.2f원 (2018)" % (campus_2022, campus_2021, campus_2018))
print("교외 (어은동, 궁동, 둔산동, 대전기타, 대전외지역) \t: %6.2f원 (2022), %6.2f원 (2021), %6.2f원 (2018)" % (outside_2022, outside_2021, outside_2018))
print("대전 (어은동, 궁동, 둔산동, 대전기타) \t\t: %6.2f원 (2022), %6.2f원 (2021), %6.2f원 (2018)" % (daejeon_2022, daejeon_2021, daejeon_2018))


2022: 7044.96원, 2021: 6282.12원, 2018: 6136.16원

[ 위치에 따른 식사 비용 ]
방 (기숙사, 집) 				: 4293.63원 (2022), 3784.06원 (2021), 4051.79원 (2018)
교내 (동측, 서측, 북측) 			: 5179.49원 (2022), 5214.29원 (2021), 4278.69원 (2018)
교외 (어은동, 궁동, 둔산동, 대전기타, 대전외지역) 	: 11138.96원 (2022), 11219.40원 (2021), 9341.18원 (2018)
대전 (어은동, 궁동, 둔산동, 대전기타) 		: 11360.00원 (2022), 9803.86원 (2021), 9153.85원 (2018)
